<p>Curso: Big Data</p>
<p>Presentado por: Boris Mauricio Martinez</p>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Lectura de datos

In [ ]:
import numpy as np
import os
import gzip

def load_data(path, type='train'):
  path_labels = os.path.join(path, f'{type}-labels-idx1-ubyte.gz')
  path_images = os.path.join(path, f'{type}-images-idx3-ubyte.gz')

  with gzip.open(path_labels, 'rb') as f_labs:
    labels = np.frombuffer(f_labs.read(), dtype=np.uint8, offset=8)
  
  with gzip.open(path_images, 'rb') as f_img:
    images = np.frombuffer(f_img.read(), dtype=np.uint8, offset=16).reshape((len(labels), 784))

  return images, labels

In [ ]:
path = '/content/drive/MyDrive/BigData/notebooks/data/fashion_mnist_data'

X_train, y_train = load_data(path=path, type='train')
X_test, y_test = load_data(path=path, type='test')

X_train = X_train[:59904, :]
X_test = X_test[:9984, :]
y_train = y_train[:59904]
y_test = y_test[:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

## Construccion de Modelo

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

tf.__version__

'2.4.1'

In [ ]:
tf.random.set_seed(200)
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5,5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5,5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0

**Ejecutar nuevamente el código anterior cada vez que se cambie el tipo de entorno de ejecución.**

## Entrenamiento con CPU

In [ ]:
import timeit

def cpu_train():
  with tf.device('/cpu:0'):
    model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

cpu_time = timeit.timeit('cpu_train()', number=1, setup='from __main__ import cpu_train')

Epoch 1/2
468/468 [==============================] - 689s 1s/step - loss: 1.0169 - accuracy: 0.7078 - val_loss: 0.4264 - val_accuracy: 0.8551
Epoch 2/2
468/468 [==============================] - 681s 1s/step - loss: 0.4105 - accuracy: 0.8562 - val_loss: 0.3127 - val_accuracy: 0.8922


In [ ]:
print('Tiempo de entrenamiento: {} segundos'.format(cpu_time))

Tiempo de entrenamiento: 1370.637937182 segundos


## Entrenamiento con GPU

In [ ]:
gpu_name = tf.test.gpu_device_name()
if gpu_name != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(gpu_name))

GPU encontrada: /device:GPU:0


In [ ]:
import timeit

def gpu_train():
  with tf.device('/device:GPU:0'):
    model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

gpu_time = timeit.timeit('gpu_train()', number=1, setup='from __main__ import gpu_train')

Epoch 1/2
468/468 [==============================] - 15s 16ms/step - loss: 1.0152 - accuracy: 0.7082 - val_loss: 0.4036 - val_accuracy: 0.8642
Epoch 2/2
468/468 [==============================] - 7s 15ms/step - loss: 0.4113 - accuracy: 0.8568 - val_loss: 0.3395 - val_accuracy: 0.8844


In [ ]:
print('Tiempo de entrenamiento: {} segundos'.format(gpu_time))

Tiempo de entrenamiento: 21.869988466999985 segundos


## Entrenamiento con TPU

### Configuración de TPU

In [ ]:
tf.random.set_seed(200)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

TPU encontrada  ['10.25.76.186:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.25.76.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.25.76.186:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with tpu_strategy.scope():
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5,5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5,5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()

  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 128)        

### Entrenamiento

In [ ]:
import timeit

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

def tpu_train():
  model.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('tpu_train()', number=1, setup='from __main__ import tpu_train')

Epoch 1/2
468/468 [==============================] - 14s 21ms/step - loss: 1.0423 - accuracy: 0.7049 - val_loss: 0.3730 - val_accuracy: 0.8726
Epoch 2/2
468/468 [==============================] - 8s 17ms/step - loss: 0.4138 - accuracy: 0.8554 - val_loss: 0.3368 - val_accuracy: 0.8806


In [ ]:
print('Tiempo de entrenamiento: {} segundos'.format(tpu_time))

Tiempo de entrenamiento: 25.12590305599997 segundos
